In [2]:
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,CenterCrop
from keras.callbacks import EarlyStopping
from keras.models import load_model
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
dir = r"./Data/Dataset_corretto"
train_set_path = r"C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Data\Train_set"
test_set_path = r"C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Data\Test_set"
valid_set_path = r"C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Data\Val_set"

In [5]:

image_size = (400, 400)
batch_size = 32
validation_split = 0.2
trainin_split = 0.8
train_test_split_ratio = 0.2 


In [6]:
datagen_1 = ImageDataGenerator(
    rescale=1./255,
      
)

In [7]:
train_set = datagen_1.flow_from_directory(
    directory=train_set_path,
    class_mode='categorical',
    color_mode="grayscale",
    target_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    interpolation="bilinear",

)


valid_set= datagen_1.flow_from_directory(
    directory = valid_set_path,
    class_mode='categorical',
    color_mode="grayscale",
    target_size=image_size,
    batch_size = batch_size,    
    shuffle=True,
    seed=42,
    interpolation="bilinear",
    
)

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'C:\\Users\\bruno\\OneDrive\\Documenti\\Deep_learning_project\\Data\\Train_set'

In [ ]:

datagen_2= ImageDataGenerator(
    rescale=1./255,
    
)


In [ ]:

test_set = datagen_2.flow_from_directory(
    directory = test_set_path,
    class_mode='categorical',
    color_mode="grayscale",
    target_size=image_size,
    batch_size = batch_size,
    shuffle=True,
    seed=42,
    interpolation="bilinear"
)



Found 12000 images belonging to 50 classes.


In [ ]:
kernel_size = (3,3)


In [ ]:
model = Sequential([
    Conv2D(filters=16, kernel_size=kernel_size, activation='relu', input_shape=(400, 400, 1)),
    MaxPooling2D(2,2),
    
    Conv2D(filters=32, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(filters=64, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(filters=128, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(filters=256, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(200, activation='relu'),
    Dropout(0.2),

    Dense(100, activation='relu'),
    Dropout(0.3), 

    Dense(50, activation='softmax')
])

In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epoche = 30


In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 398, 398, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 199, 199, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 197, 197, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 98, 98, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 96, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 48, 48, 64)       0

In [ ]:
early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
)

In [ ]:
history = model.fit(train_set, epochs=epoche, validation_data=valid_set, batch_size=batch_size,callbacks = early_stopping)


Epoch 1/30
750/750 [==============================] - 124s 155ms/step - loss: 2.5236 - accuracy: 0.2514 - val_loss: 1.6849 - val_accuracy: 0.4405
Epoch 2/30
750/750 [==============================] - 86s 115ms/step - loss: 1.7577 - accuracy: 0.4233 - val_loss: 1.2821 - val_accuracy: 0.5787
Epoch 3/30
750/750 [==============================] - 84s 112ms/step - loss: 1.3685 - accuracy: 0.5309 - val_loss: 0.9659 - val_accuracy: 0.6682
Epoch 4/30
750/750 [==============================] - 97s 129ms/step - loss: 1.0755 - accuracy: 0.6202 - val_loss: 0.7568 - val_accuracy: 0.7364
Epoch 5/30
207/750 [=======>......................] - ETA: 43s - loss: 0.8548 - accuracy: 0.6940

In [ ]:
model = load_model(r'C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\model\lego.keras')

In [ ]:

# Accuratezza
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [ ]:
# Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
save = input('vuoi salvare il modello ?')
if save == 'si':
    model.save(filepath=r'.\model\lego.keras')
    print('Modello salvato')
else: print('modello non salvato')
